## CIFAR10-using-PyTorch >>

In [57]:
#Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [58]:
#Checking for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
#Dataset setup
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## CNN Architectures >>

In [60]:
fin_dict = {"Model":[],"Accuracy":[],"Loss":[]}

In [61]:
#CNN01
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [62]:
#CNN02
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [63]:
#CNN03
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool2(nn.functional.relu(self.bn2(self.conv2(x))))
        x = self.pool3(nn.functional.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [64]:
#CNN04
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [65]:
#CNN05
class Net5(nn.Module):
    def __init__(self):
        super(Net5, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Evaluation of CNN models >>

In [66]:
# Define training function
def train(net, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01,momentum=0.9)

    net.to(device)
    net.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))

    print('Finished Training')

In [67]:
# Define evaluation function
def evaluate(net):
    criterion = nn.CrossEntropyLoss()
    net.to(device)
    net.eval()

    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    loss = running_loss / len(testloader)
    print(f"{str(net._get_name())} :")
    print(f'Accuracy: {accuracy}%, Loss: {np.round(loss,2)}%')
    fin_dict["Model"].append(net._get_name())
    fin_dict["Accuracy"].append(accuracy)
    fin_dict["Loss"].append(loss)
 

In [68]:
#Training the NNs 
for i in [Net1(), Net2(), Net3(), Net4(), Net5()]:
  netw = i
  train(i, 10)
  evaluate(netw)  

[Epoch 1] loss: 1.916
[Epoch 2] loss: 1.531
[Epoch 3] loss: 1.387
[Epoch 4] loss: 1.271
[Epoch 5] loss: 1.169
[Epoch 6] loss: 1.105
[Epoch 7] loss: 1.043
[Epoch 8] loss: 0.996
[Epoch 9] loss: 0.963
[Epoch 10] loss: 0.927
Finished Training
Net1 :
Accuracy: 69.46%, Loss: 0.88%
[Epoch 1] loss: 1.523
[Epoch 2] loss: 1.205
[Epoch 3] loss: 1.074
[Epoch 4] loss: 0.996
[Epoch 5] loss: 0.927
[Epoch 6] loss: 0.880
[Epoch 7] loss: 0.840
[Epoch 8] loss: 0.799
[Epoch 9] loss: 0.763
[Epoch 10] loss: 0.744
Finished Training
Net2 :
Accuracy: 73.93%, Loss: 0.75%
[Epoch 1] loss: 1.466
[Epoch 2] loss: 1.118
[Epoch 3] loss: 0.972
[Epoch 4] loss: 0.874
[Epoch 5] loss: 0.816
[Epoch 6] loss: 0.762
[Epoch 7] loss: 0.720
[Epoch 8] loss: 0.688
[Epoch 9] loss: 0.651
[Epoch 10] loss: 0.628
Finished Training
Net3 :
Accuracy: 75.98%, Loss: 0.69%
[Epoch 1] loss: 1.533
[Epoch 2] loss: 1.211
[Epoch 3] loss: 1.076
[Epoch 4] loss: 0.985
[Epoch 5] loss: 0.919
[Epoch 6] loss: 0.869
[Epoch 7] loss: 0.832
[Epoch 8] loss: 0.

In [69]:
report = pd.DataFrame(fin_dict)
report

,Model,Accuracy,Loss
0,Net1,69.46,0.877108
1,Net2,73.93,0.746528
2,Net3,75.98,0.688963
3,Net4,75.23,0.718233
4,Net5,73.21,0.768308
